Building a Energy Efficiency model
 

In [1]:
import pandas as pd
url = "https://raw.githubusercontent.com/StephenElston/DataScience350/master/Lecture1/EnergyEfficiencyData.csv"
df = pd.read_csv(url)

In [2]:
df.head()

,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Orientation,Glazing Area,Glazing Area Distribution,Heating Load,Cooling Load
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


In [ ]:
# ✅ 1. Step one of creating the model 
from tensorflow import keras
from tensorflow.keras import layers 

model = keras.Sequential([

    layers.Dense(units=64, activation="relu", input_shape=[8]),
    layers.Dense(units=64, activation="relu"),
    layers.Dense(units=64, activation="relu"),
    layers.Dense(units=1)
])


TypeError: Dense.__init__() missing 1 required positional argument: 'units'

In [ ]:
# ✅ 2. Compile the Model
# Before training, you need to tell Keras:
# What loss function to optimize
# What optimizer to use
# What metrics to monitor
# For a regression task (predicting continuous values):
model.compile(
    optimizer="adam",            # Adaptive optimizer
    loss="mse",                  # Mean Squared Error for regression
    metrics=["mae"]              # Mean Absolute Error as performance metric
)

📊 3. Prepare Your Data
Assuming you have your features X and target y from the energy dataset:

In [ ]:
# Side Note X should be capitalize 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Split into train and test data
X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size=0.2, random_state=42)
# Normalize inputs by scaling it to standardize it before feeding it to the model
scaler= StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) 
X_test_scaled = scaler.fit_transform(X_test)


📈 5. Evaluate the Model

In [ ]:
loss mae = model.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")